In [ ]:
# ### read the DockerFile contants #####
!cat container/Dockerfile

# Build an image that can do training and inference in SageMaker
# This is a Python 2 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

FROM ubuntu:16.04

MAINTAINER Amazon AI <sage-learner@amazon.com>


RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         python \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*

# Here we get all python packages.
# There's substantial overlap between scipy and numpy that we eliminate by
# linking them together. Likewise, pip leaves the install caches populated which uses
# a significant amount of space. These optimizations save a fair amount of space in the
# image, which reduces start up time.
RUN wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py && \
    pip install numpy scipy scikit-learn pandas cloudpickle flask gevent gunicorn && \
        (cd /usr/local/lib/python2.7/dist-packages/scipy/.libs; rm *; ln ../../numpy/.lib

In [37]:
%%sh

# The name of our algorithm
algorithm_name=alsriaz

cd container

chmod +x alsriaz/train
chmod +x alsriaz/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

# On a SageMaker Notebook Instance, the docker daemon may need to be restarted in order
# to detect your network configuration correctly.  (This is a known issue.)
if [ -d "/home/ec2-user/SageMaker" ]; then
  sudo service docker restart
fi

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}


Login Succeeded
Stopping docker: [  OK  ]
Starting docker:	.[  OK  ]

Step 1/18 : FROM ubuntu:16.04
 ---> f975c5035748
Step 2/18 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Using cache
 ---> aeddbc87b01a
Step 3/18 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 58028364b3aa
Step 4/18 : RUN wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py &&     pip install numpy scipy scikit-learn pandas cloudpickle flask gevent gunicorn &&         (cd /usr/local/lib/python2.7/dist-packages/scipy/.libs; rm *; ln ../../numpy/.libs/* .) &&         rm -rf /root/.cache
 ---> Using cache
 ---> 75306258a6e8
Step 5/18 : USER root
 ---> Using cache
 ---> d9f00b0f45e9
Step 6/18 : ENV APACHE_SPARK_VERSION 2.3.0
 ---> Using cache
 ---> 96d98af6fb8c
Step 7/18 : ENV HADOOP_VERSION 2.7
 ---> Using cache
 ---> 73a631fb8e7f
Step 8/18 : RUN apt-g

In [38]:
# S3 prefix
prefix = 'alsriaz'

# Define IAM role
import boto3
import re
import os
import numpy as np
import pandas as pd
import sagemaker as sage
from sagemaker import get_execution_role
from time import gmtime, strftime


role = get_execution_role()

sess = sage.Session()

WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/alsriaz'.format(account, region)


In [39]:
tree = sage.estimator.Estimator(image,
                       role, 1, 'ml.c4.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

In [40]:
tree.fit(data_location)

INFO:sagemaker:Creating training-job with name: alsriaz-2018-03-20-14-16-58-687


.........................................................
Traceback (most recent call last):
  File "/opt/program/train", line 13, in <module>
    import dill as pickle
ImportError: No module named dill


ValueError: Error training alsriaz-2018-03-20-14-16-58-687: Failed Reason: AlgorithmError: Exit Code: 1